In [ ]:
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import httplib2
import time
import json
import requests
import base64
import glob
import os
from os import walk
from google.colab import files

!pip install -U -q ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
import asyncio


#config
pd.set_option('display.max_columns', None)
#@markdown #***INGRESO DE USUARIO***
Dominio = '' #@param {type: "string"}
Email = '' #@param {type:"string"}
Token = ''#@param {type:"string"}
start_time="9000000" 

z_domain= Dominio
email= Email
contraseña= Token


#@markdown
authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')

headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}

carpeta_contenedora="/content/"

In [ ]:
#@markdown
url = z_domain+"/api/v2/ticket_forms"
status = 0
while status != 200:
  r = requests.get(url, headers=headers)
  
  status = r.status_code
  if status == 200:
    print("--llamada de api exitosa: ", status)
    data=json.loads(r.text)

  else:
    print("--llamada de api fracasada: ", status)
    time.sleep(10)

df_form=pd.DataFrame(data['ticket_forms'])


In [ ]:

#@markdown  Selecciona los formularios a mapear
form_available=widgets.SelectMultiple(
    options=list(df_form['name']),
    value=[],
    rows=10,
    description='Opciones',
    disabled=False
)

form_available


In [ ]:
#@markdown
def generarDataSetCamposTicket(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_fields"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    campos_de_ticket={}
    for idx, x in enumerate(data['ticket_fields']):
        id =  data['ticket_fields'][idx]['id']
        nombreCampo =  data['ticket_fields'][idx]['title']
        type =  data['ticket_fields'][idx]['type']


        if type == 'tagger' or type == 'multiselect':
            custom_field_options = data['ticket_fields'][idx]['custom_field_options']
            opciones_campo = []
            for idy, y in enumerate(custom_field_options):
                opciones_campo.append([custom_field_options[idy]['raw_name'], custom_field_options[idy]['value']])

            campos_de_ticket[id]={"name":nombreCampo, "type":type, "options":opciones_campo}

        if type == 'checkbox':
            tag = custom_field_options = data['ticket_fields'][idx]['tag']
            campos_de_ticket[id]={"name":nombreCampo, "type":type, "options": tag}

        if type != 'checkbox' and type != 'tagger' and type != 'multiselect':
            campos_de_ticket[id]={"name":nombreCampo, "type":type, "options":[]}

    
    return campos_de_ticket

form_selected=list(form_available.value)
lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)


In [ ]:
df_final=pd.DataFrame()

for index,x in df_form.iterrows():
  name=x['name']
  for gb in form_selected:
    if gb == name:
      for zx in x['ticket_field_ids']:
        #print(lista_campos[zx])
        Name_field=lista_campos[zx]["name"]
        type_field=lista_campos[zx]["type"]
        if lista_campos[zx]["options"]!=None:
          if len(lista_campos[zx]["options"])> 0:
            for xz in lista_campos[zx]["options"]:
              df_final=df_final.append({"Nombre Formulario":x['name'],"ID Campo":zx,"Nombre Campo":Name_field,"Tipo de campo":type_field,"nombre_variable":xz[0],"id_variable":xz[1]}, ignore_index = True)

          else:
            df_final=df_final.append({"Nombre Formulario":x['name'],"ID Campo":zx,"Nombre Campo":Name_field,"Tipo de campo":type_field,"nombre_variable":"","id_variable":""}, ignore_index = True)
        else:
            df_final=df_final.append({"Nombre Formulario":x['name'],"ID Campo":zx,"Nombre Campo":Name_field,"Tipo de campo":type_field,"nombre_variable":"","id_variable":""}, ignore_index = True)


df_final.to_excel("extracción.xlsx")
files.download('extracción.xlsx')
